In [2]:
import os
import json
from collections import defaultdict
from pycocotools.coco import COCO

categories = {
    'General trash':1,
    'Paper': 2,
    'Paper pack': 3,
    'Metal': 4,
    'Glass': 5,
    'Plastic': 6,
    'Styrofoam': 7,
    'Plastic bag': 8,
    'Battery': 9,
    'Clothing': 10
}

data_dir = '/opt/ml/input/data'

In [4]:
###### exclude로 지정한 이미지 제외하고 target_class로 지정한 thr보다 작은 annotation 제외하고 새로운 json 생성 #####
# 실행시간: 10초
# 사용법: 2가지만 지정해주면됨

# 변경할 json 경로 지정
file_name = 'train_sorted.json' #########지정해줄꺼 1##########
target_dir = data_dir + '/' + file_name


# 제거할 이미지 파일명 추가: 
target_class = 'General trash' #########지정해줄꺼 2##########
thr = 100
exclude = [ #########(옵션)지정해줄꺼 3##########
    
]
exclude = set(exclude)


target_json = json.load(open(target_dir))
target_coco = COCO(target_dir)
print(f'기존 이미지 개수 = {len(target_json["images"])}')
print(f'기존 annotation 개수 = {len(target_json["annotations"])}')

# 새 json 객체 생성
new_json = {
    'info': target_json['info'],
    'licenses': target_json['licenses'],
    'images': [],
    'categories': target_json['categories'],
    'annotations': [],
}

img_idx = 0 # img 인덱스 -> 순서대로 0부터 1씩 증가해야함
anno_idx = 0 # ann 인덱스 -> 이미지 상관없이 순서대로 0부터 1씩 증가해야함
print(f'시작 img 인덱스 = {img_idx}')
print(f'시작 anno 인덱스 = {anno_idx}')
removed_annos_cnt = 0
changed_annos_cnt = 0
for i in range(len(target_json['images'][:])):
    
    img_id = target_coco.getImgIds(imgIds=i)
    img_info = target_coco.loadImgs(img_id)[0]
    img_name = img_info['file_name']

    ann_ids = target_coco.getAnnIds(imgIds=img_info['id'])
    anns = target_coco.loadAnns(ann_ids)
    anns.sort(key=lambda ann:ann['area'], reverse=True) # area 큰 순

    new_json['images'].append(
        {
            'license':0,
            'url': None,
            'file_name': img_name,
            'height': 512,
            'width': 512,
            'date_captured': None,
            'id': img_idx,
        }
    )
    
    for ann in anns:
        ann['id'] = anno_idx
        ann['image_id'] = img_idx
        
        # 크기 thr보다 작은 경우 모두 target_class로 변경
        if ann['area']<thr and ann['category_id']!=categories[target_class]:
            ann['category_id'] = categories[target_class]
            changed_annos_cnt += 1

        anno_idx += 1
        new_json['annotations'].append(ann)
    img_idx += 1

print(f'마지막 img 인덱스 = {img_idx}')
print(f'마지막 anno 인덱스 = {anno_idx}')

print(f'변경된 annotation 개수 = {changed_annos_cnt}')
print(f'삭제된 annotation 개수 = {removed_annos_cnt}')

print(f'변경 후 이미지 개수 = {len(new_json["images"])}')
print(f'변경 후 annotation 개수 = {len(new_json["annotations"])}')

loading annotations into memory...
Done (t=0.96s)
creating index...
index created!
기존 이미지 개수 = 655
기존 annotation 개수 = 5252
시작 img 인덱스 = 0
시작 anno 인덱스 = 0
마지막 img 인덱스 = 655
마지막 anno 인덱스 = 5234
변경된 annotation 개수 = 55
삭제된 annotation 개수 = 18
변경 후 이미지 개수 = 655
변경 후 annotation 개수 = 5234


In [15]:
# 저장
save_path = os.path.join(data_dir,f'{file_name.split(".")[0]}_plastic_100.json')

with open(save_path,'w') as f:
        json.dump(new_json, f, indent=4)

# coco 포멧 맞는지 확인
demo = COCO(save_path)

loading annotations into memory...
Done (t=3.77s)
creating index...
index created!
